<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ резюме из HeadHunter
   

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Исследование структуры данных

1. Прочитайте данные с помощью библиотеки Pandas. Совет: перед чтением обратите внимание на разделитель внутри файла. 

In [ ]:
df_resumes = pd.read_csv('data/dst-3.0_16_1_hh_database.zip', sep=';')
print('РЕЗУЛЬТАТ ЗАДАНИЯ № 2.1:')
print('Размерность таблицы: {}'.format(df_resumes.shape))

2. Выведите несколько первых (последних) строк таблицы, чтобы убедиться в том, что ваши данные не повреждены. Ознакомьтесь с признаками и их структурой.

In [ ]:
print('РЕЗУЛЬТАТ ЗАДАНИЯ № 2.2:')
df_resumes.head()

3. Выведите основную информацию о числе непустых значений в столбцах и их типах в таблице.

4. Обратите внимание на информацию о числе непустых значений.

In [ ]:
print('РЕЗУЛЬТАТ ЗАДАНИЯ № 2.3, 2.4:')
df_resumes.info()

5. Выведите основную статистическую информацию о столбцах.


In [ ]:
print('РЕЗУЛЬТАТ ЗАДАНИЯ № 2.5:')
df_resumes.describe()

# Преобразование данных

1. Начнем с простого - с признака **"Образование и ВУЗ"**. Его текущий формат это: **<Уровень образования год выпуска ВУЗ специальность...>**. Например:
* Высшее образование 2016 Московский авиационный институт (национальный исследовательский университет)...
* Неоконченное высшее образование 2000  Балтийская государственная академия рыбопромыслового флота…
Нас будет интересовать только уровень образования.

Создайте с помощью функции-преобразования новый признак **"Образование"**, который должен иметь 4 категории: "высшее", "неоконченное высшее", "среднее специальное" и "среднее".

Выполните преобразование, ответьте на контрольные вопросы и удалите признак "Образование и ВУЗ".

Совет: обратите внимание на структуру текста в столбце **"Образование и ВУЗ"**. Гарантируется, что текущий уровень образования соискателя всегда находится в первых 2ух слов и начинается с заглавной буквы. Воспользуйтесь этим.

*Совет: проверяйте полученные категории, например, с помощью метода unique()*


In [ ]:
# временное хранилище и функция выявления количества уникальных значений образования
education_keys = set()


def check_education(value):
    education_keys.add(' '.join(value.split(' ')[:2]))


# запускаем проверку, на количество уникальных видов(типов) образовния
df_resumes['Образование и ВУЗ'].apply(check_education)

print('Виды образования (всего - {}): {}'.format(len(education_keys), education_keys))

# Cловарь соответствия старымх и новых видов образовния
education_dict = {
    'Неоконченное высшее': 'неоконченное высшее',
    'Высшее образование': 'высшее',
    'Среднее специальное': 'среднее специальное',
    'Среднее образование': 'среднее'
}

# функция обновления


def update_education(value):
    # из словаря выбираем новое значение, по ключу, сформированному из первых двух слов
    return education_dict.get(' '.join(value.split(' ')[:2]))


# запускаем пребразование, создавая новую колонку
df_resumes['Образование'] = df_resumes['Образование и ВУЗ'].apply(
    update_education)
print('Проверяем количество уникальных значений для нового признака "Образование" (должно получиться 4):{} '.format(
    df_resumes['Образование'].nunique()))

print('РЕЗУЛЬТАТ ЗАДАНИЯ № 3.1:')
print('Количество соискателей, имющих среднее образование: {}'.format(
    df_resumes[df_resumes['Образование'] == 'среднее'].shape[0]))

# удаляем колонку 'Образование и ВУЗ'
df_resumes.drop(columns=['Образование и ВУЗ'], axis=1, inplace=True)


2. Теперь нас интересует столбец **"Пол, возраст"**. Сейчас он представлен в формате **<Пол , возраст , дата рождения >**. Например:
* Мужчина , 39 лет , родился 27 ноября 1979 
* Женщина , 21 год , родилась 13 января 2000
Как вы понимаете, нам необходимо выделить каждый параметр в отдельный столбец.

Создайте два новых признака **"Пол"** и **"Возраст"**. При этом важно учесть:
* Признак пола должен иметь 2 уникальных строковых значения: 'М' - мужчина, 'Ж' - женщина. 
* Признак возраста должен быть представлен целыми числами.

Выполните преобразование, ответьте на контрольные вопросы и удалите признак **"Пол, возраст"** из таблицы.

*Совет: обратите внимание на структуру текста в столбце, в части на то, как разделены параметры пола, возраста и даты рождения между собой - символом ' , '. 
Гарантируется, что структура одинакова для всех строк в таблице. Вы можете воспользоваться этим.*


In [ ]:
# Словарь соответствия старого и нового значения пола
gender_dict = {
    'Мужчина': 'М',
    'Женщина': 'Ж'
}

# функция выделения пола из строки
def update_gender(value):
    # value.split(',') - разбиваем колонку "Пол, возраст" в массив строк, используя в качестве разделителя "," и
    # ...[0].strip() - выбираем первый элемент и удаляем окружающие пробелы
    # в качестве результат (на текущем наборе данных) должно быть или "Мужчина" или "Женщина"
    # по полученному значению(ключу) ищем новое значение в словаре
    return gender_dict[value.split(',')[0].strip()]

# функция обновления возраста
def update_age(value):
    # value.split(',') - разбиваем колонку "Пол, возраст" в массив строк, используя в качестве разделителя ","
    # ...[1].strip() - выбираем второй элемент и удаляем окружающие пробелы
    # int(...split(' ')[0].strip()) - аналогично из полученной строки получем число: разбиваем (используя " "), выбираем первое, убирем пробелы и преобразуем к числу
    return int(value.split(',')[1].strip().split(' ')[0].strip())


# создаем новые колонки используя соответсвующие функции
df_resumes['Пол'] = df_resumes['Пол, возраст'].apply(update_gender)
df_resumes['Возраст'] = df_resumes['Пол, возраст'].apply(update_age)

# вычисляем проценты резюме женщин (количества выборки по женщниам делим на общее количество)
percent_of_female = round(
    df_resumes[df_resumes['Пол'] == 'Ж'].shape[0] / df_resumes.shape[0] * 100, 2)

# вычисляем средний возраст соискателя
age_mean = round(df_resumes["Возраст"].mean(), 1)

print('РЕЗУЛЬТАТ ЗАДАНИЯ № 3.2:')
print(f'Процент женских резюме: {percent_of_female}%')
print(f'Средний возраст соискателей: {age_mean}')

# удаляем колонку 'Образование и ВУЗ'
df_resumes.drop(columns=['Пол, возраст'], axis=1, inplace=True)


3. Следующим этапом преобразуем признак **"Опыт работы"**. Его текущий формат - это: **<Опыт работы: n лет m месяцев, периоды работы в различных компаниях…>**. 

Из столбца нам необходимо выделить общий опыт работы соискателя в месяцах, новый признак назовем "Опыт работы (месяц)"

Для начала обсудим условия решения задачи:
* Во-первых, в данном признаке есть пропуски. Условимся, что если мы встречаем пропуск, оставляем его как есть (функция-преобразование возвращает NaN)
* Во-вторых, в данном признаке есть скрытые пропуски. Для некоторых соискателей в столбце стоит значения "Не указано". Их тоже обозначим как NaN (функция-преобразование возвращает NaN)
* В-третьих, нас не интересует информация, которая описывается после указания опыта работы (периоды работы в различных компаниях)
* В-четвертых, у нас есть проблема: опыт работы может быть представлен только в годах или только месяцах. Например, можно встретить следующие варианты:
    * Опыт работы 3 года 2 месяца…
    * Опыт работы 4 года…
    * Опыт работы 11 месяцев…
    * Учитывайте эту особенность в вашем коде

Учитывайте эту особенность в вашем коде

В результате преобразования у вас должен получиться столбец, содержащий информацию о том, сколько месяцев проработал соискатель.
Выполните преобразование, ответьте на контрольные вопросы и удалите столбец **"Опыт работы"** из таблицы.


In [ ]:
# определяем ключевые фразы
key_words_year = ('год', 'года', 'лет')
key_words_month = ('месяц', 'месяца', 'месяцев')

# функция обновления опыта
def update_work_experience(value):
    # Если NaN или 'Не указано', возвращаем NaN, иначе обрабатываем
    if value is np.nan or value == 'Не указано':
        return np.nan
    else:
        # инициализируем переменные
        years, months = 0, 0

        # Разбиваем строку и берем первые 6 элементов (макс.возможная строка, исходя из фразы "Опыт работы ХХ лет YY месяцев")
        worlds = value.split(' ')[:6]
        # Перебираем каждое слово на предмет его вхождения во временный диапазон (год или месяц)
        for i in range(len(worlds)):
            # Если слово связано с 'год', то предыдущее значение преобразуем в числовой год
            if worlds[i] in key_words_year:
                years = int(worlds[i - 1])
            # Если слово связано с 'месяц', то предыдущее слово преобразуем в числовой месяца
            if worlds[i] in key_words_month:
                months = int(worlds[i - 1])
        return years * 12 + months


# Создаем новую колонку, используя функцию добавляем туда опыт работы в месяцах
df_resumes['Опыт работы (месяц)'] = df_resumes['Опыт работы'].apply(
    update_work_experience)

# Формируем вывод
print('РЕЗУЛЬТАТ ЗАДАНИЯ № 3.3:')
print('Медианный опыт работы (в месяцах): {}'.format(
    df_resumes['Опыт работы (месяц)'].median()))

# Удаляем колонку 'Опыт работы'
df_resumes.drop(columns=['Опыт работы'], axis=1, inplace=True)


4. Хорошо идем! Следующий на очереди признак "Город, переезд, командировки". Информация в нем представлена в следующем виде: **<Город , (метро) , готовность к переезду (города для переезда) , готовность к командировкам>**. В скобках указаны необязательные параметры строки. Например, можно встретить следующие варианты:

* Москва , не готов к переезду , готов к командировкам
* Москва , м. Беломорская , не готов к переезду, не готов к командировкам
* Воронеж , готов к переезду (Сочи, Москва, Санкт-Петербург) , готов к командировкам

Создадим отдельные признаки **"Город"**, **"Готовность к переезду"**, **"Готовность к командировкам"**. При этом важно учесть:

* Признак **"Город"** должен содержать только 4 категории: "Москва", "Санкт-Петербург" и "город-миллионник" (их список ниже), остальные обозначьте как "другие".

    Список городов-миллионников:
    
   <code>million_cities = ['Новосибирск', 'Екатеринбург','Нижний Новгород','Казань', 'Челябинск','Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж','Волгоград']
    </code>
    Инфорация о метро, рядом с которым проживает соискатель нас не интересует.
* Признак **"Готовность к переезду"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к переезду в признаке "Город, переезд, командировки". Например:
    * … , готов к переезду , …
    * … , не готова к переезду , …
    * … , готова к переезду (Москва, Санкт-Петербург, Ростов-на-Дону)
    * … , хочу переехать (США) , …
    
    Нас интересует только сам факт возможности или желания переезда.
* Признак **"Готовность к командировкам"** должен иметь два возможных варианта: True или False. Обратите внимание, что возможны несколько вариантов описания готовности к командировкам в признаке "Город, переезд, командировки". Например:
    * … , готов к командировкам , … 
    * … , готова к редким командировкам , …
    * … , не готов к командировкам , …
    
    Нас интересует только сам факт готовности к командировке.
    
    Еще один важный факт: при выгрузки данных у некоторых соискателей "потерялась" информация о готовности к командировкам. Давайте по умолчанию будем считать, что такие соискатели не готовы к командировкам.
    
Выполните преобразования и удалите столбец **"Город, переезд, командировки"** из таблицы.

*Совет: обратите внимание на то, что структура текста может меняться в зависимости от указания ближайшего метро. Учите это, если будете использовать порядок слов в своей программе.*


In [ ]:
# словарь, для городов сохранюящих свое название
citieis_keep_name = ['Москва', 'Санкт-Петербург']
# словарь, для городов миллиоников
million_cities = ['Новосибирск', 'Екатеринбург', 'Нижний Новгород', 'Казань', 'Челябинск',
                  'Омск', 'Самара', 'Ростов-на-Дону', 'Уфа', 'Красноярск', 'Пермь', 'Воронеж', 'Волгоград']

# функция очистки от данных которые в скобках(включая скобки"()"), считаем,
# что в данном случае данные в скобках "лишние", но могут влиять на резлуьтат,
# т.к. содержат запятые, символы разделения основных ("нужных") данных
def clear_brackets(value):
    brackets = []
    result = []
    for c in value:
        if c == '(':
            brackets.append(c)
        elif c == ')':
            if len(brackets) > 0 and brackets[-1] == '(':
                brackets = brackets[:-1]
        elif len(brackets) == 0:
            result.append(c)
    return ''.join(result)


# функция обновления города:
# - выбираем первое слово (город) из строки и пошагово выбираем конечное значение
def update_city(value):
    value = value.split(',')[0].strip()
    if value in citieis_keep_name:
        return value
    elif value in million_cities:
        return 'город-миллионник'
    else:
        return 'другие'


# функция разбора данных и получения ключевой фразы для готовности(неготовности) к переезду(is_moving=True)/командировкам(is_moving=False)
def get_moving_or_transfer_key_word(value, is_moving=True):
    # очищаем от данных в скобках
    values = clear_brackets(value).strip().split(',')

    # выбираем фразу(по умолчанию), готовности к перемещению
    key_moving = values[1]
    # выбираем фразу(по умолчанию), готовности к коммандировкам, с учетом того, что ее может не быть
    key_transfer = values[2] if len(values) > 2 else 'не готов'
    # проверяем наличия метро во второй колонке, и при наличии метро обновляем фразы
    if 'м.' in values[1]:
        key_moving = values[2]
        key_transfer = values[3] if len(values) > 3 else 'не готов'
    return key_moving.strip() if is_moving else key_transfer.strip()


# функция обновления готовности к переезду
def update_workg_moving(value):
    result = get_moving_or_transfer_key_word(value, is_moving=True)
    return False if len(result) == 0 or 'не готов' in result else True

# функция обновления готовности к командировкам
def update_work_travel(value):
    result = get_moving_or_transfer_key_word(value, is_moving=False)
    return False if len(result) == 0 or 'не готов' in result else True


# создание новых колонок с использованием функций
df_resumes['Город'] = df_resumes['Город, переезд, командировки'].apply(
    update_city)
df_resumes['Готовность к переезду'] = df_resumes['Город, переезд, командировки'].apply(
    update_workg_moving)
df_resumes['Готовность к командировкам'] = df_resumes['Город, переезд, командировки'].apply(
    update_work_travel)

# вывод результата
print('РЕЗУЛЬТАТ ЗАДАНИЯ № 3.4:')
print('Количество соискателей из Санкт-Петербурга(%): {}'.format(round(
    df_resumes[df_resumes['Город'] == 'Санкт-Петербург']['Город'].count() / df_resumes.shape[0] * 100)))
print('Количество соискателей готовых к переезду и к командировкам(%): {}'.format(round(
    df_resumes[df_resumes['Готовность к переезду'] & df_resumes['Готовность к командировкам']]['Город'].count() / df_resumes.shape[0] * 100)))

# Удаляем колонку 'Город, переезд, командировки'
df_resumes.drop(columns=['Город, переезд, командировки'], axis=1, inplace=True)

5. Рассмотрим поближе признаки **"Занятость"** и **"График"**. Сейчас признаки представляют собой набор категорий желаемой занятости (полная занятость, частичная занятость, проектная работа, волонтерство, стажировка) и желаемого графика работы (полный день, сменный график, гибкий график, удаленная работа, вахтовый метод).
На сайте hh.ru соискатель может указывать различные комбинации данных категорий, например:
* полная занятость, частичная занятость
* частичная занятость, проектная работа, волонтерство
* полный день, удаленная работа
* вахтовый метод, гибкий график, удаленная работа, полная занятость

Такой вариант признаков имеет множество различных комбинаций, а значит множество уникальных значений, что мешает анализу. Нужно это исправить!

Давайте создадим признаки-мигалки для каждой категории: если категория присутствует в списке желаемых соискателем, то в столбце на месте строки рассматриваемого соискателя ставится True, иначе - False.

Такой метод преобразования категориальных признаков называется One Hot Encoding и его схема представлена на рисунке ниже:
<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/ohe.jpg>
Выполните данное преобразование для признаков "Занятость" и "График", ответьте на контрольные вопросы, после чего удалите их из таблицы

In [ ]:
# формируем словари, встречающихся значений "График" и "Занятость"
research_cols = {
    'График': set(),
    'Занятость': set()
}

# функция заполнения уникальных значений для признаков "График" и "Занятость"
def fill_research_cols(key, value):
    values = value.split(',')
    for token in values:
        research_cols[key].add(token.strip())
    return False


# Перебираем каждый ключевой признак
for col in research_cols:
    # Заполняем уникальные значений для этого признака, используя временную колонку
    df_resumes.apply(lambda row: fill_research_cols(col, row[col]) ,axis=1)
    # Выводим (для проверки) все возможные значения для этого признака
    print('Возможные варианты для признака(колонки) "{}" (всего-{}): {}'.format(col, len(research_cols[col]), research_cols[col]))
    
    # Перебираем каждое значение из возможных для исследуемого признака
    for col_value in research_cols[col]:
        # Cоздаем (и заполняем) новую колонку для каждого из возможных значений исходного признака (One Hot Encoding),
        # устанавливаем True, если исследуемое значение присутствует в значении исходного признака, иначе False
        df_resumes[col_value] = df_resumes.apply(lambda row2: True if col_value in row2[col] else False, axis=1)


# выодим результат (количество строк), фильтруя по необходимым параметрам
print()
print('РЕЗУЛЬТАТ ЗАДАНИЯ № 3.5:')
print('Количество людей, которые ищут `проектную работу` и `волонтёрство` одновременно: {}'.format(df_resumes[df_resumes['волонтерство'] & df_resumes['проектная работа']].shape[0]))
print('Количество людей, которые хотят работать `вахтовым методом` и с `гибким графиком` одновременно: {}'.format(df_resumes[df_resumes['вахтовый метод'] & df_resumes['гибкий график']].shape[0]))

# Удаляем "ненужные" колонки
df_resumes.drop(columns=['График'], axis=1, inplace=True)
df_resumes.drop(columns=['Занятость'], axis=1, inplace=True)

6. (2 балла) Наконец, мы добрались до самого главного и самого важного - признака заработной платы **"ЗП"**. 
В чем наша беда? В том, что помимо желаемой заработной платы соискатель указывает валюту, в которой он бы хотел ее получать, например:
* 30000 руб.
* 50000 грн.
* 550 USD

Нам бы хотелось видеть заработную плату в единой валюте, например, в рублях. Возникает вопрос, а где взять курс валют по отношению к рублю?

На самом деле язык Python имеет в арсенале огромное количество возможностей получения данной информации, от обращения к API Центробанка, до использования специальных библиотек, например pycbrf. Однако, это не тема нашего проекта.

Поэтому мы пойдем в лоб: обратимся к специальным интернет-ресурсам для получения данных о курсе в виде текстовых файлов. Например, MDF.RU, данный ресурс позволяет удобно экспортировать данные о курсах различных валют и акций за указанные периоды в виде csv файлов. Мы уже сделали выгрузку курсов валют, которые встречаются в наших данных за период с 29.12.2017 по 05.12.2019. Скачать ее вы можете **на платформе**

Создайте новый DataFrame из полученного файла. В полученной таблице нас будут интересовать столбцы:
* "currency" - наименование валюты в ISO кодировке,
* "date" - дата, 
* "proportion" - пропорция, 
* "close" - цена закрытия (последний зафиксированный курс валюты на указанный день).


Перед вами таблица соответствия наименований иностранных валют в наших данных и их общепринятых сокращений, которые представлены в нашем файле с курсами валют. Пропорция - это число, за сколько единиц валюты указан курс в таблице с курсами. Например, для казахстанского тенге курс на 20.08.2019 составляет 17.197 руб. за 100 тенге, тогда итоговый курс равен - 17.197 / 100 = 0.17197 руб за 1 тенге.
Воспользуйтесь этой информацией в ваших преобразованиях.

<img src=https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/table.jpg>


Осталось только понять, откуда брать дату, по которой определяется курс? А вот же она - в признаке **"Обновление резюме"**, в нем содержится дата и время, когда соискатель выложил текущий вариант своего резюме. Нас интересует только дата, по ней бы и будем сопоставлять курсы валют.

Теперь у нас есть вся необходимая информация для того, чтобы создать признак "ЗП (руб)" - заработная плата в рублях.

После ответа на контрольные вопросы удалите исходный столбец заработной платы "ЗП" и все промежуточные столбцы, если вы их создавали.

Итак, давайте обсудим возможный алгоритм преобразования: 
1. Перевести признак "Обновление резюме" из таблицы с резюме в формат datetime и достать из него дату. В тот же формат привести признак "date" из таблицы с валютами.
2. Выделить из столбца "ЗП" сумму желаемой заработной платы и наименование валюты, в которой она исчисляется. Наименование валюты перевести в стандарт ISO согласно с таблицей выше.
3. Присоединить к таблице с резюме таблицу с курсами по столбцам с датой и названием валюты (подумайте, какой тип объединения надо выбрать, чтобы в таблице с резюме сохранились данные о заработной плате, изначально представленной в рублях). Значение close для рубля заполнить единицей 1 (курс рубля самого к себе)
4. Умножить сумму желаемой заработной платы на присоединенный курс валюты (close) и разделить на пропорцию (обратите внимание на пропуски после объединения в этих столбцах), результат занести в новый столбец "ЗП (руб)".


In [ ]:
# загрузка основных курсов валюты за исследуемый период
df_exchange_rates = pd.read_csv('data/ExchangeRates.csv')


# Ниже в вспомогательный участок кода, помогающий выявить возможные варианты
# наименования валюты в существующих данных, для проверки, что ничего не упустили
currency_name_in_data = set()


def fill_currency_name_in_data(value):
    currency_name_in_data.add(value.split(' ')[1].strip())

df_resumes['ЗП'].apply(fill_currency_name_in_data)

print('Возможные варианты валюты в текущих данных (всего-{}):{}'.format(
    len(currency_name_in_data), currency_name_in_data))

# Получили следуюище данные: {'бел.руб.', 'EUR', 'USD', 'KGS', 'KZT', 'сум', 'грн.', 'AZN', 'руб.'}.
# На их основе и справочных данных из задания, формируется следующий словарь соответствия:
dict_currency_relation = {'бел.руб.': 'BYN', 'EUR': 'EUR', 'USD': 'USD', 'KGS': 'KGS',
                          'KZT': 'KZT', 'сум': 'UZS', 'грн.': 'UAH', 'AZN': 'AZN', 'руб.': 'RUB'}

# Функция разбивки признака ЗП на два признака: значение (количество) ЗП и валюта ЗП
def split_salary(value, type):
    values = value.split(' ')
    if type == 0:
        return int(values[0].strip())
    else:
        return dict_currency_relation[values[1].strip()]


# Используя функцию 'split_salary', создаем и заполняем новые признаки: значения и валюты ЗП
df_resumes['ЗП_значение'] = df_resumes.apply(
    lambda row: split_salary(row['ЗП'], 0), axis=1)
df_resumes['ЗП_валюта'] = df_resumes.apply(
    lambda row: split_salary(row['ЗП'], 1), axis=1)

# В каждой из таблиц обновляем колонку даты, для связывания даты актуальности резюме и актуального курса валюты
df_exchange_rates['date'] = pd.to_datetime(
    df_exchange_rates['date'], dayfirst=True).dt.date
df_resumes['Обновление резюме'] = pd.to_datetime(
    df_resumes['Обновление резюме'], dayfirst=True).dt.date


# Объединяем таблицы, по дате обмена и типу валюты, объединение "LEFT" (ВСЕ записи из таблицы резюме),
# т.к. обмена рублей на рубли нету
df_resumes = df_resumes.merge(
    df_exchange_rates,
    left_on=['Обновление резюме', 'ЗП_валюта'],
    right_on=['date', 'currency'],
    how='left'
)

# Заполняем отсутствующие значения для рублевых ЗП
df_resumes.fillna(
    {
        'close': 1,
        'proportion': 1
    },
    inplace=True
)
# Вычисляем ЗП в рублях (новая колонка), используя курс и пропорцию
df_resumes['ЗП (руб)'] = df_resumes['ЗП_значение'] * \
    df_resumes['close'] / df_resumes['proportion']

# выодим результат (количество строк), фильтруя по необходимым параметрам
print()
print('РЕЗУЛЬТАТ ЗАДАНИЯ № 3.6:')
print('Желаемая медианная заработная плата соискателей (в т.рублей): {}'.format(
    round(df_resumes['ЗП (руб)'].median()/1000)))

# Удаляем ненужные колонки: исходные, временно созданные и смерженные из таблицы курсов
df_resumes.drop(columns=['ЗП'], axis=1, inplace=True)
df_resumes.drop(columns=['ЗП_значение'], axis=1, inplace=True)
df_resumes.drop(columns=['ЗП_валюта'], axis=1, inplace=True)
for col in df_exchange_rates.columns:
    df_resumes.drop(columns=[col], axis=1, inplace=True)


# Исследование зависимостей в данных

1. Постройте распределение признака **"Возраст"**. Опишите распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится возраст большинства соискателей? Есть ли аномалии для признака возраста, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
# Определение функции для вывода двух диаграмм: гистограммы и коробчатой, для заданий 4.1, 4.2, 4.3
def define_and_show_diagramms(data_feature):
    diagram_x_value = data_feature
    diagram_x_range = [0, df_resumes[diagram_x_value].max()]
    diagram_x_mode = round(df_resumes[diagram_x_value].mode()[0])

    fig = px.histogram(
        df_resumes,
        x=diagram_x_value,
        text_auto=True,
        range_x=diagram_x_range,
        title='Гистограмма распределения количества резюме в зависимости от признака "{}" соискателя  (мода распределения для признака "{}" - {})'.format(diagram_x_value, diagram_x_value, diagram_x_mode)
    )
    fig.update_layout(yaxis_title='Резюме (кол-во)')
    fig.add_vline(x=diagram_x_mode)
    fig.show()

    fig = px.box(
        df_resumes,
        x=diagram_x_value,
        range_x=diagram_x_range,
        title='Выявление граничащих значений по признаку "{}"'.format(diagram_x_value)
    )
    fig.update_layout(yaxis_title='Резюме (кол-во)')
    fig.show()

# прорисовка диаграмм для признака 'Возраст'
define_and_show_diagramms('Возраст')

### Выводы
**Вопрос**: Чему равна мода распределения?

**Ответ:** 30 лет

**Вопрос**: Каковы предельные значения признака, в каком примерном интервале находится возраст большинства соискателей? 

**Ответ:**: Предельные значения: мин - 14, мах - 100 лет, примерный интервал большинства соисткателей: от 27 до 36 лет

**Вопрос:** Есть ли аномалии для данного признака? Если есть, то какие значения вы бы причислили к таковым?

**Ответ:**: Из графика к потенциальным аномалиям можно отнести резюме с возрастом от 50 лет и выше, из жизненого опыта к аномалиям можно отнести резюме с возрастом свыше 70 лет

2. Постройте распределение признака **"Опыт работы (месяц)"**. Опишите данное распределение, отвечая на следующие вопросы: чему равна мода распределения, каковы предельные значения признака, в каком примерном интервале находится опыт работы большинства соискателей? Есть ли аномалии для признака опыта работы, какие значения вы бы причислили к их числу?
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*

In [ ]:
# прорисовка диаграмм для признака 'Опыт работы (месяц)'
define_and_show_diagramms('Опыт работы (месяц)')

### Выводы
**Вопрос:** Чему равна мода распределения?

**Ответ:** Мода распределения равна 81 месяц.


**Вопрос:** Каковы предельные значения признака, в каком примерном интервале находится опыт работы большинства соискателей?

**Ответ:** Предельные значения: от 1(один) до 1188 месяцев, опыт работы большинства в интервале от 57 до 154 месяцев.

**Вопрос:** Есть ли аномалии для признака? Если есть, то какие значения вы бы причислили к таковым?

**Ответ:** Аномалии есть, по графику они начинаются с 300 месяцев опыта, из жизненого опыта, я бы к аномалиям причислял опыт свыше 564 месяцев (макс.выход на пенсию 65 лет, начинать работать можно после школы 18 лет): 65-18=47 лет работы * 12 месяцев = 564 месяца

3. Постройте распределение признака **"ЗП (руб)"**. Опишите данное распределение, отвечая на следующие вопросы: каковы предельные значения признака, в каком примерном интервале находится заработная плата большинства соискателей? Есть ли аномалии для признака возраста? Обратите внимание на гигантские размеры желаемой заработной платы.
*Совет: постройте гистограмму и коробчатую диаграмму рядом.*


In [ ]:
# прорисовка диаграмм для признака 'ЗП (руб)'
define_and_show_diagramms('ЗП (руб)')

### Выводы

**Постройте распределение признака «ЗП (руб)». Опишите распределение, отвечая на следующие вопросы:**

**Вопрос:** Чему равна мода распределения?

**Ответ:** Мода распределения равна **50000**.

**Вопрос:** Каковы предельные значения признака, в каком примерном интервале находится заработная плата большинства соискателей?

**Ответ:** Предельные значения: от 1(один) до 24 304 880 руб, пожелания большинства от 1 (один) до 180 900.5 руб.

**Вопрос:** Есть ли аномалии для признака заработной платы? Если есть, то какие значения вы бы причислили к таковым?


**Ответ:** Аномалии есть, по графику они начинаются с 180 900.5 руб, я бы к аномалиям причислял ЗП свыше 5 млн. рублей

4. Постройте диаграмму, которая показывает зависимость **медианной** желаемой заработной платы (**"ЗП (руб)"**) от уровня образования (**"Образование"**). Используйте для диаграммы данные о резюме, где желаемая заработная плата меньше 1 млн рублей.
*Сделайте выводы по представленной диаграмме: для каких уровней образования наблюдаются наибольшие и наименьшие уровни желаемой заработной платы? Как вы считаете, важен ли признак уровня образования при прогнозировании заработной платы?*

In [ ]:
# Определяем ключевые признаки
diagram_x_value = 'ЗП (руб)'
diagram_y_value = 'Образование'


# Создаем фильтар (ЗП < 1 млн.)
filter_salary_lt_million = df_resumes[diagram_x_value] < 1_000_000

# Рисуем столбчатую диаграмму
fig = px.bar(
    df_resumes[filter_salary_lt_million].groupby(by=diagram_y_value, as_index=False)[diagram_x_value].median(),
    x=diagram_x_value,
    y=diagram_y_value,
    title='Зависимость медианной желаемой заработной платы («ЗП (руб)») от уровня образования («Образование»).'
)
fig.show()

### Выводы

**Вопрос:** Для каких уровней образования наблюдаются наибольшие и наименьшие уровни желаемой заработной платы?

**Ответ:** Наименьшее для средне специального и среднего, наибольшее для высшего

**Вопрос:** Как вы считаете, важен ли признак уровня образования при прогнозировании заработной платы?

**Ответ:** Признак "Образование" **важен** при прогнозировании ЗП

5. Постройте диаграмму, которая показывает распределение желаемой заработной платы (**"ЗП (руб)"**) в зависимости от города (**"Город"**). Используйте для диаграммы данные о резюме, где желая заработная плата меньше 1 млн рублей.
*Сделайте выводы по полученной диаграмме: как соотносятся медианные уровни желаемой заработной платы и их размах в городах? Как вы считаете, важен ли признак города при прогнозировании заработной платы?*

In [ ]:
# Определяем ключевые признаки
diagram_x_value = 'ЗП (руб)'
diagram_y_value = 'Город'

fig = px.box(
    df_resumes[filter_salary_lt_million].sort_values(by=diagram_y_value),
    x=diagram_x_value,
    y=diagram_y_value,
    color=diagram_y_value
)
fig.show()

### Выводы

**Вопрос:** Как соотносятся медианные уровни желаемой заработной платы и их размах в городах?

**Ответ:** Как и ожидалось в Москве наиболее ожидаемый уровнеь ЗП (1 место), Санкт-Петербург на 2-ом, города-миллионники и другие разделяют 3 и 4 места

**Вопрос:** Как вы считаете, важен ли признак города при прогнозировании заработной платы?

**Ответ:** Признак "Город" **важен** при прогнозировании ЗП.

6. Постройте **многоуровневую столбчатую диаграмму**, которая показывает зависимость медианной заработной платы (**"ЗП (руб)"**) от признаков **"Готовность к переезду"** и **"Готовность к командировкам"**. Проанализируйте график, сравнив уровень заработной платы в категориях.

In [ ]:
# Определяем ключевые признаки
diagram_x_value = ['Готовность к командировкам', 'Готовность к переезду']
diagram_y_value = 'ЗП (руб)'

fig = px.bar(
    df_resumes.groupby(by=diagram_x_value, as_index=False)[diagram_y_value].median(),
    x=diagram_x_value[0],
    y=diagram_y_value,
    color=diagram_x_value[1]
)
fig.show()

### Выводы
**Задание:** Проанализируйте график, сравнив уровень заработной платы по категориям.

1) Самая низкая ЗП у тех кто **НЕ готов** к переезду и **НЕ готов** к командировкам
2) Самая высокая ЗП у тех кто **готов** к пеерезду и **готов** к командировкам
3) У оставшихся, кто **готов** к чему то одному, ЗП больше у тех, кто **готов** к коммандировкам

7. Постройте сводную таблицу, иллюстрирующую зависимость **медианной** желаемой заработной платы от возраста (**"Возраст"**) и образования (**"Образование"**). На полученной сводной таблице постройте **тепловую карту**. Проанализируйте тепловую карту, сравнив показатели внутри групп.

In [ ]:
# Строим сводную таблицу и по ней гисуем тепловую карту
fig = px.imshow(
    df_resumes.pivot_table(index='Образование', columns='Возраст', values='ЗП (руб)', aggfunc='median'),
    text_auto=True
)
fig.show()

### Выводы
**Задание:** Проанализируйте тепловую карту, сравнив показатели внутри групп

Зарплата "повышается" с повышением уровня образования и повышением возраста (опыта работы?).
При чем зарплата "повышается" быстрее у более высокого уровня образования.

8. Постройте **диаграмму рассеяния**, показывающую зависимость опыта работы (**"Опыт работы (месяц)"**) от возраста (**"Возраст"**). Опыт работы переведите из месяцев в года, чтобы признаки были в едином масштабе. Постройте на графике дополнительно прямую, проходящую через точки (0, 0) и (100, 100). Данная прямая соответствует значениям, когда опыт работы равен возрасту человека. Точки, лежащие на этой прямой и выше нее - аномалии в наших данных (опыт работы больше либо равен возрасту соискателя)

In [ ]:
# Добавляем временную колонку - опыт работы в годах
df_resumes['Опыт работы (год)'] = df_resumes['Опыт работы (месяц)'] / 12
fig = px.scatter(
    df_resumes,
    y = 'Опыт работы (год)',
    x = 'Возраст',
    width=1500,
    height=1000
)
fig.update_layout(shapes=[
    dict(
      type= 'line',
      yref= 'y', y0= 0, y1= 100,
      xref= 'x', x0= 0, x1= 100
    )
])
fig.show()

# Удаляяем временную колонку
df_resumes.drop(columns=['Опыт работы (год)'], axis=1, inplace=True)

### Выводы
1) прямая по диагонали хорошо выявляет аномалии, т.к. не может опыт работы быть больше возраста (нельзя начать работаь еще не родившись)
2) можно выявит еще больше аномалий, "сдвинув" прямую вправо на 10-17 лет, в зависимости от того с какого возраста можно начинать учитывать опыт работы
3) в основ массе данные выглядят правдоподнобно "толстая" диагональ слева направо, кто-то начинал работаь раньше, кто-то позднее

**Дополнительные баллы**

Для получения 2 дополнительных баллов по разведывательному анализу постройте еще два любых содержательных графика или диаграммы, которые помогут проиллюстрировать влияние признаков/взаимосвязь между признаками/распределения признаков. Приведите выводы по ним. Желательно, чтобы в анализе участвовали признаки, которые мы создавали ранее в разделе "Преобразование данных".


In [ ]:
# ваш код здесь

ваши выводы здесь

# Очистка данных

1. Начнем с дубликатов в наших данных. Найдите **полные дубликаты** в таблице с резюме и удалите их. 

In [ ]:
print('РЕЗУЛЬТАТ ЗАДАНИЯ № 5.1:')
print('Количество полных дубликатов: {}'.format(df_resumes[df_resumes.duplicated()].shape[0]))
df_resumes.drop_duplicates(inplace=True)

2. Займемся пропусками. Выведите информацию **о числе пропусков** в столбцах. 

In [ ]:
print('РЕЗУЛЬТАТ ЗАДАНИЯ № 5.2:')
display(df_resumes.info())
print('Количество пропусков в столбце "Опыт работы (месяц)": {}'.format(df_resumes['Опыт работы (месяц)'].isnull().sum()))

3. Итак, у нас есть пропуски в 3ех столбцах: **"Опыт работы (месяц)"**, **"Последнее/нынешнее место работы"**, **"Последняя/нынешняя должность"**. Поступим следующим образом: удалите строки, где есть пропуск в столбцах с местом работы и должностью. Пропуски в столбце с опытом работы заполните **медианным** значением.

In [ ]:
# Заполняем пропуски в признаке 'Опыт работы (месяц)' медианным значением
df_resumes.fillna(
    {
        'Опыт работы (месяц)': df_resumes['Опыт работы (месяц)'].median()
    },
    inplace=True
)
# Удаляем строки где есть хотя бы одно пустое значение, это 'Последнее/нынешнее место работы' и 'Последняя/нынешняя должность'
df_resumes.dropna(thresh=df_resumes.shape[1], axis=0, inplace=True)
print('РЕЗУЛЬТАТ ЗАДАНИЯ № 5.3:')
print('Среднее значение в столбце "Опыт работы (месяц)" после заполнения пропусков: {}'.format(round(df_resumes['Опыт работы (месяц)'].mean())))

4. Мы добрались до ликвидации выбросов. Сначала очистим данные вручную. Удалите резюме, в которых указана заработная плата либо выше 1 млн. рублей, либо ниже 1 тыс. рублей.

In [ ]:
# Условие поиска некорректной ЗП
salary_query = (df_resumes['ЗП (руб)'] > 1_000_000) | (df_resumes['ЗП (руб)'] < 1_000)

print('РЕЗУЛЬТАТ ЗАДАНИЯ № 5.4:')
print('Количество строк с некорректной ЗП (меньше тысячи и больше миллиона рублей): {}'.format(df_resumes[salary_query].shape[0]))

# Удаляем некорректную ЗП
df_resumes.drop(df_resumes[salary_query].index, axis=0, inplace=True)

5. В процессе разведывательного анализа мы обнаружили резюме, в которых **опыт работы в годах превышал возраст соискателя**. Найдите такие резюме и удалите их из данных


In [ ]:
# Условие поиска некорректного возраста и опыта работы
age_query = df_resumes['Опыт работы (месяц)'] > df_resumes['Возраст'] * 12
print('РЕЗУЛЬТАТ ЗАДАНИЯ № 5.5:')
print('Количество людей у которых "Опыт работы (месяц)" больше "Возраст": {}'.format(df_resumes[age_query].shape[0]))

# Удаляем записи, где "Опыт работы (месяц)" больше "Возраст"
df_resumes.drop(df_resumes[age_query].index, axis=0, inplace=True)

6. В результате анализа мы обнаружили потенциальные выбросы в признаке **"Возраст"**. Это оказались резюме людей чересчур преклонного возраста для поиска работы. Попробуйте построить распределение признака в **логарифмическом масштабе**. Добавьте к графику линии, отображающие **среднее и границы интервала метода трех сигм**. Напомним, сделать это можно с помощью метода axvline. Например, для построение линии среднего будет иметь вид:

`histplot.axvline(log_age.mean(), color='k', lw=2)`

В какую сторону асимметрично логарифмическое распределение? Напишите об этом в комментарии к графику.
Найдите выбросы с помощью метода z-отклонения и удалите их из данных, используйте логарифмический масштаб. Давайте сделаем послабление на **1 сигму** (возьмите 4 сигмы) в **правую сторону**.

Выведите таблицу с полученными выбросами и оцените, с каким возрастом соискатели попадают под категорию выбросов?

In [ ]:
# Рисуем логарифмическое распределение  по возрасту
fig, ax = plt.subplots(1, 1, figsize=(8, 4))
df_resumes_log = np.log(df_resumes['Возраст'])
histplot = sns.histplot(df_resumes_log, bins=30, ax=ax)
histplot.axvline(df_resumes_log.mean(), color='k', lw=2)
histplot.axvline(df_resumes_log.mean() + 3 *
                 df_resumes_log.std(), color='k', ls='--', lw=2)
histplot.axvline(df_resumes_log.mean() - 3 *
                 df_resumes_log.std(), color='k', ls='--', lw=2)
histplot.set_title('Логарифмическое распределение по признаку "Возраст"')
fig.show()

# функция для удаления выбросов, определенных с помощью метода N(3)-сигм


def outliers_z_score_mod(data, feature, left=3, right=3, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - left * sigma
    upper_bound = mu + right * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned


outliers, cleaned = outliers_z_score_mod(df_resumes, 'Возраст', 3, 4, True)
display(outliers['Возраст'])


### Выводы
1) Логарифмическое распределение ассиметрично вправо
2) Под категорию выбросов попадают соискатели с возратом 15 и 100 лет